In [ ]:
"""installs"""

# !pip install nc-time-axis
# !pip install netCDF4 h5netcdf
# !pip install xarray
# !pip install cdsapi
# !pip install networkx geopandas osmnx igraph matplotlib


'installs'

In [ ]:
import sys

sys.path.append("../")


In [ ]:
# Core libraries
import networkx as nx
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Optional
from dataclasses import dataclass


from scripts import model

UTM_CRS = 32639

CRS = 4326
DEMAND_COL = "demand_without"
SERVICE_NAME = "hospital"
SETTL_NAME = "nao"


# Constants
KELVIN_TO_CELSIUS = 273.15
BASE_YEAR = 2025
FORECAST_YEAR = 2050
YEAR_STEP = 5
PROBABILITY_THRESHOLD = 0.5


CONST_BASE_DEMAND = 120  # if not known (!)
SERVICE_RADIUS_MINUTES = 300

transport_modes = [
    "car_warm",
    "plane",
    "water_ship",
    "water_boat",
    "car_cold",
    "winter_tr",
]


In [ ]:
main_data_path = "../data"


settl = gpd.read_file(
    main_data_path + f"/processed/df_settlements_{SETTL_NAME}.geojson"
).to_crs(CRS)
transport_df = (
    gpd.read_file(main_data_path + f"/processed/df_time_{SETTL_NAME}.geojson")
    .to_crs(CRS)
    .dropna(subset=["geometry"])
)

df_service = gpd.read_file(
    main_data_path + f"/processed/df_{SERVICE_NAME}_{SETTL_NAME}.geojson"
).to_crs(UTM_CRS)

infr_df = pd.read_csv(main_data_path + f"/raw/infrastructure_{SETTL_NAME}.csv", sep=";")


## Some preprocessing

In [ ]:
settl["name"] = settl["name"].str.replace("ё", "е")
# Convert mode values from ',' to '.' and ensure they are float
for mode in transport_modes:
    transport_df[mode] = (
        transport_df[mode].astype(str).str.replace(",", ".").astype(float)
    )

# Fill NaNs with 0
infr_df.fillna(0, inplace=True)

for t in transport_modes:
    transport_df[t] = transport_df[t].astype(str).str.replace(",", ".").astype(float)


In [ ]:
x = transport_df.union_all().centroid.x
y = transport_df.union_all().centroid.y


In [ ]:
# import folium
# from folium import FeatureGroup
# import geopandas as gpd


# # Start a base map
# m = folium.Map(location=[x, y], zoom_start=5, tiles="cartodbpositron")

# # Define transport columns you want to show as layers
# transport_cols = [
#     "car_warm",
#     "plane",
#     "water_ship",
#     "water_boat",
#     "car_cold",
#     "winter_tr",
# ]

# # Color palette just for better visibility
# colors = ["red", "blue", "green", "purple", "orange", "black"]

# # Create a layer for each transport type
# for mode, color in zip(transport_cols, colors):
#     fg = FeatureGroup(name=mode)

#     # Filter edges with non-zero availability for this transport type
#     gdf_filtered = transport_df[transport_df[mode] > 0]

#     # Add each geometry as a line to the FeatureGroup
#     for _, row in gdf_filtered.iterrows():
#         folium.PolyLine(
#             locations=[(coord[1], coord[0]) for coord in row["geometry"].coords],
#             color=color,
#             tooltip=f"{row['edge1']} ↔ {row['edge2']} ({mode})",
#             weight=3,
#         ).add_to(fg)

#     fg.add_to(m)
# m


In [ ]:
# ??????????????????

df_matrix = model.make_adjacency_matrix(transport_df)
blocks_gdf = model.create_blocks(settl, const_demand=CONST_BASE_DEMAND, epsg=UTM_CRS)
matrix = model.create_graph(df_matrix)
blocks = model.update_blocks_with_services(
    blocks_gdf, df_service, epsg=UTM_CRS, service_type_name=SERVICE_NAME
)
id_matrix = df_matrix.index.tolist()


In [ ]:
@dataclass
class TransportMode:
    name: str
    winter_threshold: float = -10.0
    summer_threshold: float = 5.0
    steepness: float = 0.3

    def get_probability(self, temperature: float) -> float:
        """Calculate probability using sigmoid function"""
        return 1 / (1 + np.exp(-self.steepness * (temperature - self.winter_threshold)))


class CarWarmTransport(TransportMode):
    def __init__(self):
        super().__init__(name="car_warm", winter_threshold=5.0, steepness=0.3)


class WinterTransport(TransportMode):
    def __init__(self):
        super().__init__(name="winter_tr", winter_threshold=-5.0, steepness=-0.4)


class PlaneTransport(TransportMode):
    def __init__(self):
        super().__init__(name="plane", winter_threshold=-5.0, steepness=0.3)


class WaterTransport(TransportMode):
    def __init__(self):
        super().__init__(name="water_ship", winter_threshold=0.0, steepness=0.7)


TRANSPORT_MODES = [
    CarWarmTransport(),
    WinterTransport(),
    PlaneTransport(),
    WaterTransport(),
]
